In [1]:
using ConcreteStructs

@concrete struct Foo a; b; c end

macro with_kwargs(typename::Symbol)
    T = Core.eval(__module__, typename)
    names = fieldnames(T)
    kwargs = Expr(:parameters, names...)
    :($typename($kwargs) = $typename($(names...))) |> esc
end

macro with_kwargs(typename::Symbol, kwargs_expr)
    T = Core.eval(__module__, typename)
    names = fieldnames(T)
    D = Core.eval(__module__, kwargs_expr)
    kwargs = Expr(:parameters)
    for name in names
        val = get(D, name, nothing)
        if isnothing(val)
            push!(kwargs.args, name)
        else
            push!(kwargs.args, Expr(:kw, name, val))
        end
    end
    :($typename($kwargs) = $typename($(names...))) |> esc
end


@with_kwargs (macro with 2 methods)

In [2]:
@macroexpand @with_kwargs Foo

:(Foo(; a, b, c) = begin
          #= In[1]:9 =#
          Foo(a, b, c)
      end)

In [3]:
@with_kwargs Foo

Foo

In [4]:
methods(Foo)

# 2 methods for type constructor:
[1] Foo(; a, b, c) in Main at In[1]:9
[2] Foo(a::__T_a, b::__T_b, c::__T_c) where {__T_a, __T_b, __T_c} in Main at D:\.julia\packages\ConcreteStructs\kcpNb\src\ConcreteStructs.jl:141

In [5]:
Foo(a = 1, b = 2.0, c = "three")

Foo{Int64, Float64, String}(1, 2.0, "three")

In [6]:
@macroexpand @with_kwargs Foo (a = 1, c = "three")

:(Foo(; a = 1, b, c = "three") = begin
          #= In[1]:25 =#
          Foo(a, b, c)
      end)

In [7]:
@with_kwargs Foo (a = 1, c = "three")

Foo

In [8]:
methods(Foo)

# 2 methods for type constructor:
[1] Foo(; a, b, c) in Main at In[1]:25
[2] Foo(a::__T_a, b::__T_b, c::__T_c) where {__T_a, __T_b, __T_c} in Main at D:\.julia\packages\ConcreteStructs\kcpNb\src\ConcreteStructs.jl:141

In [9]:
Foo(b = 2.0)

Foo{Int64, Float64, String}(1, 2.0, "three")

In [10]:
default = (a = 1, b = 2.0, c = "three")
@macroexpand @with_kwargs Foo default

:(Foo(; a = 1, b = 2.0, c = "three") = begin
          #= In[1]:25 =#
          Foo(a, b, c)
      end)

In [11]:
@with_kwargs Foo default

Foo

In [12]:
methods(Foo)

# 2 methods for type constructor:
[1] Foo(; a, b, c) in Main at In[1]:25
[2] Foo(a::__T_a, b::__T_b, c::__T_c) where {__T_a, __T_b, __T_c} in Main at D:\.julia\packages\ConcreteStructs\kcpNb\src\ConcreteStructs.jl:141

In [13]:
Foo()

Foo{Int64, Float64, String}(1, 2.0, "three")